### Notebook for tf.eager()

In [1]:
import numpy
import tensorflow as tf
tfe = tf.contrib.eager # Abbreviation. 
tf.enable_eager_execution()
print('TF Eager Execution is:','Enabled' if tf.executing_eagerly() else 'Disabled')

TF Eager Execution is: Enabled


In [2]:
tfe.Variable([1,2,3,4])

<tf.Variable 'Variable:0' shape=(4,) dtype=int32, numpy=array([1, 2, 3, 4], dtype=int32)>

In [9]:
## This works with side_lengths as a sequence and numpy array but not
## With a TensorFlow eager Variable
def f4(side_lengths): 
    
    #t_a,t_b,t_c,t_d = side_lengths not working
    
    t_a = side_lengths[0]
    t_b = side_lengths[1]
    t_c = side_lengths[2]
    t_d = side_lengths[3]
    
    t_ab = t_a * t_b
    t_cd = t_c * t_d
    t_abc = t_ab * t_c
    t_abd = t_ab * t_d
    t_acd = t_a * t_cd
    t_bcd = t_b * t_cd
    t_s = t_abc + t_abd + t_acd + t_bcd 
    ret = 2.0 * t_s
    return ret 
print(f4([1.0, 2.0, 3.0, 4.0]), f4(numpy.array([1.0, 2.0, 3.0, 4.0])))
p0 = tfe.Variable([1.0, 2.0, 3.0, 4.0])
f4(p0).numpy()

100.0 100.0


100.0

In [4]:
## Alternative version
def f4(abcd):
    return 2.0 * sum(abcd[i] * abcd[j] * abcd[k] 
                     for (i, j, k) in [(0, 1, 2), (0, 1, 3),
                                       (0, 2, 3), (1, 2, 3)])

print(f4([1.0, 2.0, 3.0, 4.0]), f4(numpy.array([1.0, 2.0, 3.0, 4.0])))
p0 = tfe.Variable([1.0, 2.0, 3.0, 4.0])
f4(p0)

100.0 100.0


<tf.Tensor: id=101, shape=(), dtype=float32, numpy=100.0>

In [39]:
with tf.GradientTape() as tape:
    hypersurface = f4(p0)

In [40]:
gradients_at_p0 = tape.gradient(hypersurface, [p0])
gradients_at_p0

[<tf.Tensor: id=934, shape=(4,), dtype=float32, numpy=array([52., 38., 28., 22.], dtype=float32)>]

In [41]:
print('Hypersurface:', hypersurface.numpy(),
      'Gradient:', gradients_at_p0[0].numpy())

Hypersurface: 100.0 Gradient: [52. 38. 28. 22.]


In [42]:
p1 = tfe.Variable([1.0, 3.0, 3.0, 4.0])

In [38]:
tape.gradient(hypersurface, [p1])

RuntimeError: GradientTape.gradient can only be called once on non-persistent tapes.

In [50]:
tape2 = tf.GradientTape()
with tape2: #<-- here we record on the tape
    hypersurface1 = f4(p0)
    hypersurface2 = f4(2 * p0)
    total_hypersurface = hypersurface1 + hypersurface2
    for i in range(10):
        total_hypersurface += total_hypersurface

#<--- here we can access the tape 

In [51]:
tape2.gradient(total_hypersurface, p0) #468., 342., 252., 198

<tf.Tensor: id=1901, shape=(4,), dtype=float32, numpy=array([479232., 350208., 258048., 202752.], dtype=float32)>